<h1>导入数据</h1>

In [56]:
import matplotlib.pyplot as plt
import pandas as pd
import time
from datetime import datetime
from tqdm import tqdm
from time import strftime, localtime
from scipy import stats
from scipy.spatial import distance
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.utils.class_weight import compute_class_weight
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score
from sklearn.model_selection import GridSearchCV
from lightgbm import LGBMClassifier
from joblib import dump, load
from tensorflow.keras.backend import epsilon
from itertools import combinations
from imblearn.over_sampling import SMOTE, ADASYN
from imblearn.ensemble import RUSBoostClassifier, BalancedBaggingClassifier

<h1>数据处理</h1>

<h2>数据预处理</h2>

<h3>conf_info数据预处理</h3>

In [3]:
cons_info = pd.read_csv("../data/训练组_比特币挖矿_档案明细（20211220）.csv", index_col=0, quotechar='"', encoding="gbk")
cons_info

,ID,ELEC_TYPE_NAME,VOLT_NAME,PRC_NAME,CONTRACT_CAP,RUN_CAP,SHIFT_NO,BUILD_DATE,CANCEL_DATE,CHK_CYCLE,LAST_CHK_DATE,TMP_NAME,TMP_DATE,IS_FLAG
,,,,,,,,,,,,,,
1,329465205,乡村居民生活用电,交流220V,居民合表电价(不满1千伏）,4.0,4.0,1.0,2005/6/18,NaN,240.0,2005/6/18,非临时用电,NaN,0
2,329465245,乡村居民生活用电,交流220V,居民合表电价(不满1千伏）,4.0,4.0,1.0,2005/6/18,NaN,240.0,2005/6/18,非临时用电,NaN,0
3,329465357,乡村居民生活用电,交流220V,居民合表电价(不满1千伏）,8.0,8.0,1.0,2005/6/18,NaN,240.0,2005/6/18,非临时用电,NaN,0
4,329465360,乡村居民生活用电,交流220V,居民合表电价(不满1千伏）,2.0,2.0,1.0,2005/6/18,NaN,240.0,2005/6/18,非临时用电,NaN,0
5,329465439,乡村居民生活用电,交流220V,居民合表电价(不满1千伏）,8.0,8.0,1.0,2005/6/18,NaN,240.0,2005/6/18,非临时用电,NaN,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8836,2862711473,乡村居民生活用电,交流380V,居民合表电价(不满1千伏）,39.0,39.0,NaN,2019/12/2 16:12:00,NaN,120.0,2019/12/2 16:12:00,非临时用电,NaN,0
8837,2864274650,乡村居民生活用电,交流220V,居民合表电价(不满1千伏）,13.0,13.0,NaN,2019/12/3 16:33:00,NaN,120.0,2019/12/3 16:33:00,非临时用电,NaN,0
8838,2872313739,乡村居民生活用电,交流380V,居民合表电价(不满1千伏）,39.0,39.0,NaN,2019/12/18 14:42:00,NaN,120.0,2019/12/18 14:42:00,非临时用电,NaN,0


In [4]:
cons_info = cons_info.set_index("ID").drop(columns=["CANCEL_DATE", "TMP_DATE"])
cons_info

,ELEC_TYPE_NAME,VOLT_NAME,PRC_NAME,CONTRACT_CAP,RUN_CAP,SHIFT_NO,BUILD_DATE,CHK_CYCLE,LAST_CHK_DATE,TMP_NAME,IS_FLAG
ID,,,,,,,,,,,
329465205,乡村居民生活用电,交流220V,居民合表电价(不满1千伏）,4.0,4.0,1.0,2005/6/18,240.0,2005/6/18,非临时用电,0
329465245,乡村居民生活用电,交流220V,居民合表电价(不满1千伏）,4.0,4.0,1.0,2005/6/18,240.0,2005/6/18,非临时用电,0
329465357,乡村居民生活用电,交流220V,居民合表电价(不满1千伏）,8.0,8.0,1.0,2005/6/18,240.0,2005/6/18,非临时用电,0
329465360,乡村居民生活用电,交流220V,居民合表电价(不满1千伏）,2.0,2.0,1.0,2005/6/18,240.0,2005/6/18,非临时用电,0
329465439,乡村居民生活用电,交流220V,居民合表电价(不满1千伏）,8.0,8.0,1.0,2005/6/18,240.0,2005/6/18,非临时用电,0
...,...,...,...,...,...,...,...,...,...,...,...
2862711473,乡村居民生活用电,交流380V,居民合表电价(不满1千伏）,39.0,39.0,NaN,2019/12/2 16:12:00,120.0,2019/12/2 16:12:00,非临时用电,0
2864274650,乡村居民生活用电,交流220V,居民合表电价(不满1千伏）,13.0,13.0,NaN,2019/12/3 16:33:00,120.0,2019/12/3 16:33:00,非临时用电,0
2872313739,乡村居民生活用电,交流380V,居民合表电价(不满1千伏）,39.0,39.0,NaN,2019/12/18 14:42:00,120.0,2019/12/18 14:42:00,非临时用电,0


In [5]:
cons_info["BUILD_DATE"] = pd.to_datetime(cons_info["BUILD_DATE"])
cons_info["LAST_CHK_DATE"] = pd.to_datetime(cons_info["LAST_CHK_DATE"])
cons_info

,ELEC_TYPE_NAME,VOLT_NAME,PRC_NAME,CONTRACT_CAP,RUN_CAP,SHIFT_NO,BUILD_DATE,CHK_CYCLE,LAST_CHK_DATE,TMP_NAME,IS_FLAG
ID,,,,,,,,,,,
329465205,乡村居民生活用电,交流220V,居民合表电价(不满1千伏）,4.0,4.0,1.0,2005-06-18 00:00:00,240.0,2005-06-18 00:00:00,非临时用电,0
329465245,乡村居民生活用电,交流220V,居民合表电价(不满1千伏）,4.0,4.0,1.0,2005-06-18 00:00:00,240.0,2005-06-18 00:00:00,非临时用电,0
329465357,乡村居民生活用电,交流220V,居民合表电价(不满1千伏）,8.0,8.0,1.0,2005-06-18 00:00:00,240.0,2005-06-18 00:00:00,非临时用电,0
329465360,乡村居民生活用电,交流220V,居民合表电价(不满1千伏）,2.0,2.0,1.0,2005-06-18 00:00:00,240.0,2005-06-18 00:00:00,非临时用电,0
329465439,乡村居民生活用电,交流220V,居民合表电价(不满1千伏）,8.0,8.0,1.0,2005-06-18 00:00:00,240.0,2005-06-18 00:00:00,非临时用电,0
...,...,...,...,...,...,...,...,...,...,...,...
2862711473,乡村居民生活用电,交流380V,居民合表电价(不满1千伏）,39.0,39.0,NaN,2019-12-02 16:12:00,120.0,2019-12-02 16:12:00,非临时用电,0
2864274650,乡村居民生活用电,交流220V,居民合表电价(不满1千伏）,13.0,13.0,NaN,2019-12-03 16:33:00,120.0,2019-12-03 16:33:00,非临时用电,0
2872313739,乡村居民生活用电,交流380V,居民合表电价(不满1千伏）,39.0,39.0,NaN,2019-12-18 14:42:00,120.0,2019-12-18 14:42:00,非临时用电,0


In [6]:
cons_info["SHIFT_NO"] = cons_info["SHIFT_NO"].fillna(0)
cons_info

,ELEC_TYPE_NAME,VOLT_NAME,PRC_NAME,CONTRACT_CAP,RUN_CAP,SHIFT_NO,BUILD_DATE,CHK_CYCLE,LAST_CHK_DATE,TMP_NAME,IS_FLAG
ID,,,,,,,,,,,
329465205,乡村居民生活用电,交流220V,居民合表电价(不满1千伏）,4.0,4.0,1.0,2005-06-18 00:00:00,240.0,2005-06-18 00:00:00,非临时用电,0
329465245,乡村居民生活用电,交流220V,居民合表电价(不满1千伏）,4.0,4.0,1.0,2005-06-18 00:00:00,240.0,2005-06-18 00:00:00,非临时用电,0
329465357,乡村居民生活用电,交流220V,居民合表电价(不满1千伏）,8.0,8.0,1.0,2005-06-18 00:00:00,240.0,2005-06-18 00:00:00,非临时用电,0
329465360,乡村居民生活用电,交流220V,居民合表电价(不满1千伏）,2.0,2.0,1.0,2005-06-18 00:00:00,240.0,2005-06-18 00:00:00,非临时用电,0
329465439,乡村居民生活用电,交流220V,居民合表电价(不满1千伏）,8.0,8.0,1.0,2005-06-18 00:00:00,240.0,2005-06-18 00:00:00,非临时用电,0
...,...,...,...,...,...,...,...,...,...,...,...
2862711473,乡村居民生活用电,交流380V,居民合表电价(不满1千伏）,39.0,39.0,0.0,2019-12-02 16:12:00,120.0,2019-12-02 16:12:00,非临时用电,0
2864274650,乡村居民生活用电,交流220V,居民合表电价(不满1千伏）,13.0,13.0,0.0,2019-12-03 16:33:00,120.0,2019-12-03 16:33:00,非临时用电,0
2872313739,乡村居民生活用电,交流380V,居民合表电价(不满1千伏）,39.0,39.0,0.0,2019-12-18 14:42:00,120.0,2019-12-18 14:42:00,非临时用电,0


In [7]:
cons_info["now"] = "2021-12-31"
cons_info["now"] = pd.to_datetime(cons_info["now"])
cons_info

,ELEC_TYPE_NAME,VOLT_NAME,PRC_NAME,CONTRACT_CAP,RUN_CAP,SHIFT_NO,BUILD_DATE,CHK_CYCLE,LAST_CHK_DATE,TMP_NAME,IS_FLAG,now
ID,,,,,,,,,,,,
329465205,乡村居民生活用电,交流220V,居民合表电价(不满1千伏）,4.0,4.0,1.0,2005-06-18 00:00:00,240.0,2005-06-18 00:00:00,非临时用电,0,2021-12-31
329465245,乡村居民生活用电,交流220V,居民合表电价(不满1千伏）,4.0,4.0,1.0,2005-06-18 00:00:00,240.0,2005-06-18 00:00:00,非临时用电,0,2021-12-31
329465357,乡村居民生活用电,交流220V,居民合表电价(不满1千伏）,8.0,8.0,1.0,2005-06-18 00:00:00,240.0,2005-06-18 00:00:00,非临时用电,0,2021-12-31
329465360,乡村居民生活用电,交流220V,居民合表电价(不满1千伏）,2.0,2.0,1.0,2005-06-18 00:00:00,240.0,2005-06-18 00:00:00,非临时用电,0,2021-12-31
329465439,乡村居民生活用电,交流220V,居民合表电价(不满1千伏）,8.0,8.0,1.0,2005-06-18 00:00:00,240.0,2005-06-18 00:00:00,非临时用电,0,2021-12-31
...,...,...,...,...,...,...,...,...,...,...,...,...
2862711473,乡村居民生活用电,交流380V,居民合表电价(不满1千伏）,39.0,39.0,0.0,2019-12-02 16:12:00,120.0,2019-12-02 16:12:00,非临时用电,0,2021-12-31
2864274650,乡村居民生活用电,交流220V,居民合表电价(不满1千伏）,13.0,13.0,0.0,2019-12-03 16:33:00,120.0,2019-12-03 16:33:00,非临时用电,0,2021-12-31
2872313739,乡村居民生活用电,交流380V,居民合表电价(不满1千伏）,39.0,39.0,0.0,2019-12-18 14:42:00,120.0,2019-12-18 14:42:00,非临时用电,0,2021-12-31


In [8]:
cons_info["live_days"] = (cons_info["now"] - cons_info["BUILD_DATE"]).map(lambda x: x.days)
cons_info["check_days"] = (cons_info["now"] - cons_info["LAST_CHK_DATE"]).map(lambda x: x.days)
cons_info

,ELEC_TYPE_NAME,VOLT_NAME,PRC_NAME,CONTRACT_CAP,RUN_CAP,SHIFT_NO,BUILD_DATE,CHK_CYCLE,LAST_CHK_DATE,TMP_NAME,IS_FLAG,now,live_days,check_days
ID,,,,,,,,,,,,,,
329465205,乡村居民生活用电,交流220V,居民合表电价(不满1千伏）,4.0,4.0,1.0,2005-06-18 00:00:00,240.0,2005-06-18 00:00:00,非临时用电,0,2021-12-31,6040,6040
329465245,乡村居民生活用电,交流220V,居民合表电价(不满1千伏）,4.0,4.0,1.0,2005-06-18 00:00:00,240.0,2005-06-18 00:00:00,非临时用电,0,2021-12-31,6040,6040
329465357,乡村居民生活用电,交流220V,居民合表电价(不满1千伏）,8.0,8.0,1.0,2005-06-18 00:00:00,240.0,2005-06-18 00:00:00,非临时用电,0,2021-12-31,6040,6040
329465360,乡村居民生活用电,交流220V,居民合表电价(不满1千伏）,2.0,2.0,1.0,2005-06-18 00:00:00,240.0,2005-06-18 00:00:00,非临时用电,0,2021-12-31,6040,6040
329465439,乡村居民生活用电,交流220V,居民合表电价(不满1千伏）,8.0,8.0,1.0,2005-06-18 00:00:00,240.0,2005-06-18 00:00:00,非临时用电,0,2021-12-31,6040,6040
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2862711473,乡村居民生活用电,交流380V,居民合表电价(不满1千伏）,39.0,39.0,0.0,2019-12-02 16:12:00,120.0,2019-12-02 16:12:00,非临时用电,0,2021-12-31,759,759
2864274650,乡村居民生活用电,交流220V,居民合表电价(不满1千伏）,13.0,13.0,0.0,2019-12-03 16:33:00,120.0,2019-12-03 16:33:00,非临时用电,0,2021-12-31,758,758
2872313739,乡村居民生活用电,交流380V,居民合表电价(不满1千伏）,39.0,39.0,0.0,2019-12-18 14:42:00,120.0,2019-12-18 14:42:00,非临时用电,0,2021-12-31,743,743


In [9]:
cons_info = cons_info.drop(columns=["BUILD_DATE", "LAST_CHK_DATE", "now"])
cons_info

,ELEC_TYPE_NAME,VOLT_NAME,PRC_NAME,CONTRACT_CAP,RUN_CAP,SHIFT_NO,CHK_CYCLE,TMP_NAME,IS_FLAG,live_days,check_days
ID,,,,,,,,,,,
329465205,乡村居民生活用电,交流220V,居民合表电价(不满1千伏）,4.0,4.0,1.0,240.0,非临时用电,0,6040,6040
329465245,乡村居民生活用电,交流220V,居民合表电价(不满1千伏）,4.0,4.0,1.0,240.0,非临时用电,0,6040,6040
329465357,乡村居民生活用电,交流220V,居民合表电价(不满1千伏）,8.0,8.0,1.0,240.0,非临时用电,0,6040,6040
329465360,乡村居民生活用电,交流220V,居民合表电价(不满1千伏）,2.0,2.0,1.0,240.0,非临时用电,0,6040,6040
329465439,乡村居民生活用电,交流220V,居民合表电价(不满1千伏）,8.0,8.0,1.0,240.0,非临时用电,0,6040,6040
...,...,...,...,...,...,...,...,...,...,...,...
2862711473,乡村居民生活用电,交流380V,居民合表电价(不满1千伏）,39.0,39.0,0.0,120.0,非临时用电,0,759,759
2864274650,乡村居民生活用电,交流220V,居民合表电价(不满1千伏）,13.0,13.0,0.0,120.0,非临时用电,0,758,758
2872313739,乡村居民生活用电,交流380V,居民合表电价(不满1千伏）,39.0,39.0,0.0,120.0,非临时用电,0,743,743


In [10]:
cons_info.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8840 entries, 329465205 to 2529558569
Data columns (total 11 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   ELEC_TYPE_NAME  8840 non-null   object 
 1   VOLT_NAME       8840 non-null   object 
 2   PRC_NAME        8840 non-null   object 
 3   CONTRACT_CAP    8840 non-null   float64
 4   RUN_CAP         8840 non-null   float64
 5   SHIFT_NO        8840 non-null   float64
 6   CHK_CYCLE       8838 non-null   float64
 7   TMP_NAME        8840 non-null   object 
 8   IS_FLAG         8840 non-null   int64  
 9   live_days       8840 non-null   int64  
 10  check_days      8840 non-null   int64  
dtypes: float64(4), int64(3), object(4)
memory usage: 828.8+ KB


In [11]:
cons_info = pd.get_dummies(cons_info)
cons_info

,CONTRACT_CAP,RUN_CAP,SHIFT_NO,CHK_CYCLE,IS_FLAG,live_days,check_days,ELEC_TYPE_NAME_乡村居民生活用电,ELEC_TYPE_NAME_商业用电,ELEC_TYPE_NAME_城镇居民生活用电,ELEC_TYPE_NAME_学校教学和学生生活用电,ELEC_TYPE_NAME_居民生活用电,ELEC_TYPE_NAME_普通工业,ELEC_TYPE_NAME_非居民照明,ELEC_TYPE_NAME_非工业,VOLT_NAME_交流10kV,VOLT_NAME_交流220V,VOLT_NAME_交流380V,PRC_NAME_居民合表电价(1-10千伏）,PRC_NAME_居民合表电价(不满1千伏）,TMP_NAME_非临时用电
ID,,,,,,,,,,,,,,,,,,,,,
329465205,4.0,4.0,1.0,240.0,0,6040,6040,1,0,0,0,0,0,0,0,0,1,0,0,1,1
329465245,4.0,4.0,1.0,240.0,0,6040,6040,1,0,0,0,0,0,0,0,0,1,0,0,1,1
329465357,8.0,8.0,1.0,240.0,0,6040,6040,1,0,0,0,0,0,0,0,0,1,0,0,1,1
329465360,2.0,2.0,1.0,240.0,0,6040,6040,1,0,0,0,0,0,0,0,0,1,0,0,1,1
329465439,8.0,8.0,1.0,240.0,0,6040,6040,1,0,0,0,0,0,0,0,0,1,0,0,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2862711473,39.0,39.0,0.0,120.0,0,759,759,1,0,0,0,0,0,0,0,0,0,1,0,1,1
2864274650,13.0,13.0,0.0,120.0,0,758,758,1,0,0,0,0,0,0,0,0,1,0,0,1,1
2872313739,39.0,39.0,0.0,120.0,0,743,743,1,0,0,0,0,0,0,0,0,0,1,0,1,1


In [12]:
cons_info["CHK_CYCLE"] = cons_info["CHK_CYCLE"].fillna(cons_info["CHK_CYCLE"].median())
cons_info.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8840 entries, 329465205 to 2529558569
Data columns (total 21 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   CONTRACT_CAP                8840 non-null   float64
 1   RUN_CAP                     8840 non-null   float64
 2   SHIFT_NO                    8840 non-null   float64
 3   CHK_CYCLE                   8840 non-null   float64
 4   IS_FLAG                     8840 non-null   int64  
 5   live_days                   8840 non-null   int64  
 6   check_days                  8840 non-null   int64  
 7   ELEC_TYPE_NAME_乡村居民生活用电     8840 non-null   uint8  
 8   ELEC_TYPE_NAME_商业用电         8840 non-null   uint8  
 9   ELEC_TYPE_NAME_城镇居民生活用电     8840 non-null   uint8  
 10  ELEC_TYPE_NAME_学校教学和学生生活用电  8840 non-null   uint8  
 11  ELEC_TYPE_NAME_居民生活用电       8840 non-null   uint8  
 12  ELEC_TYPE_NAME_普通工业         8840 non-null   uint8  
 13  ELEC_TYPE_NAME_非居民照

<h3>elec_month数据预处理</h3>

In [13]:
cons_info["pq_z_mean"] = 0
cons_info["pq_z_median"] = 0
cons_info["pq_z_std"] = 0
cons_info["pq_z_std/mean"] = 0
cons_info["pq_z_entropy"] = 0
cons_info["pq_fg_distance"] = 0
cons_info["pq_gp_diatance"] = 0
cons_info["pq_fp_distance"] = 0
cons_info["pq_fg_cosine"] = 0
cons_info["pq_gp_cosine"] = 0
cons_info["pq_fp_cosine"] = 0
cons_info["pq_fz_cosine"] = 0
cons_info["pq_gz_cosine"] = 0
cons_info["pq_pz_cosine"] = 0
cons_info["variation_month"] = 0
cons_info["pq_z_mean_4500"] = 0
cons_info["variation_month_0085"] = 0

cons_info["pq_z_max"] = 0
cons_info["pq_z_min"] = 0

cons_info["entropy_month"] = 0
cons_info["std_month"] = 0

cons_info["diff_month"] = 0
cons_info["devide_month"] = 0

In [14]:
def entropy(data, bins: int = 10) -> float:
    """
    计算一个序列的熵\n
    """
    nums = np.histogram(data, bins=bins)[0]
    probs = nums / sum(nums)
    return stats.entropy(probs)

In [15]:
def entropy_df(df: pd.DataFrame, bins: int = 10) -> float:
    """
    计算一段时间子电量的熵\n
    """
    entropys = df.apply(lambda x: entropy(x), axis=1)
    return entropys.mean()

In [16]:
def std_df(df: pd.DataFrame) -> float:
    """
    计算一段时间子电量的标准差\n
    """
    return df.std(axis=1).mean()

In [17]:
def cosine(a, b) -> float:
    """
    计算余弦距离\n
    """
    if np.linalg.norm(a) != 0 and np.linalg.norm(b) != 0:
        return distance.cosine(a, b)
    elif np.linalg.norm(a) == 0 and np.linalg.norm(b) == 0:
        return 0
    else:
        return 1

In [18]:
def diff_median(df: pd.DataFrame) -> float:
    """
    计算一个df的两列数值差的中位数\n
    """
    results = []
    for i, j in combinations(list(range(df.shape[1])), r=2):
        result = (df.iloc[:, i] - df.iloc[:, j]).apply(lambda x: abs(x)).median()
        results.append(result)
    return sum(results)

In [19]:
def devide_median(df: pd.DataFrame) -> float:
    """
    计算两个列的相除的中位数\n
    """
    results = []
    for i, j in combinations(list(range(df.shape[1])), r=2):
        result = (df.iloc[:, 0] / (df.iloc[:, 1] + epsilon())).median()
        results.append(result)
    return sum(results)

In [20]:
def variation(df: pd.DataFrame) -> float:
    """
    计算变异系数\n
    """
    tt =  (epsilon() + df.std(axis=1)) / (epsilon() + df.mean(axis=1))
    tt[tt.apply(lambda x: np.isinf(x))] = 0
    return tt.mean(axis=0)

In [21]:
elec_month = pd.read_csv("../data/训练组_比特币挖矿_月用电明细（20211217）.csv")
elec_month

,id,ym,pq_f,pq_g,pq_p,pq_z
0,329769614,202001,615,161,0,776
1,329769614,202002,492,154,-1,645
2,329769614,202003,457,114,1,572
3,329769614,202004,522,171,0,693
4,329769614,202005,519,181,0,700
...,...,...,...,...,...,...
194475,2878334859,202106,107,67,-1,173
194476,2878334859,202107,170,112,1,283
194477,2878334859,202108,102,67,0,169
194478,2878334859,202109,112,65,-1,176


In [22]:
elec_month["ym"] = elec_month["ym"].astype(str)
elec_month

,id,ym,pq_f,pq_g,pq_p,pq_z
0,329769614,202001,615,161,0,776
1,329769614,202002,492,154,-1,645
2,329769614,202003,457,114,1,572
3,329769614,202004,522,171,0,693
4,329769614,202005,519,181,0,700
...,...,...,...,...,...,...
194475,2878334859,202106,107,67,-1,173
194476,2878334859,202107,170,112,1,283
194477,2878334859,202108,102,67,0,169
194478,2878334859,202109,112,65,-1,176


In [23]:
for cons_id in tqdm(cons_info.index):
    df = elec_month[elec_month["id"] == cons_id]
    df = df.drop(columns=["id"]).sort_values("ym").set_index("ym")
    df = df.fillna(method="ffill").fillna(method="bfill")
    cons_info.loc[cons_id, "pq_z_mean"] = df["pq_z"].mean()
    cons_info.loc[cons_id, "pq_z_median"] = df["pq_z"].median()
    cons_info.loc[cons_id, "pq_z_std"] = df["pq_z"].std()
    cons_info.loc[(cons_info.index == cons_id) & (cons_info["pq_z_mean"] > 0), "pq_z_std/mean"] = cons_info["pq_z_std"] / cons_info["pq_z_mean"]
    cons_info.loc[cons_id, "pq_z_entropy"] = entropy(df["pq_z"])
    
    cons_info.loc[cons_id, "pq_fg_distance"] = distance.euclidean(df["pq_f"], df["pq_g"])
    cons_info.loc[cons_id, "pq_gp_diatance"] = distance.euclidean(df["pq_g"], df["pq_p"])
    cons_info.loc[cons_id, "pq_fp_distance"] = distance.euclidean(df["pq_f"], df["pq_p"])
    
    cons_info.loc[cons_id, "pq_fg_cosine"] = cosine(df["pq_f"], df["pq_g"])
    cons_info.loc[cons_id, "pq_gp_cosine"] = cosine(df["pq_g"], df["pq_p"])
    cons_info.loc[cons_id, "pq_fp_cosine"] = cosine(df["pq_f"], df["pq_p"])
    cons_info.loc[cons_id, "pq_fz_cosine"] = cosine(df["pq_f"], df["pq_z"])
    cons_info.loc[cons_id, "pq_gz_cosine"] = cosine(df["pq_g"], df["pq_z"])
    cons_info.loc[cons_id, "pq_pz_cosine"] = cosine(df["pq_p"], df["pq_z"])
    
    cons_info.loc[cons_id, "variation_month"] = variation(df[["pq_f", "pq_g", "pq_p"]])
    cons_info.loc[cons_id, "pq_z_mean_4500"] = (cons_info.loc[cons_id, "pq_z_mean"] > 4500).astype(float)
    cons_info.loc[cons_id, "variation_month_0085"] = (cons_info.loc[cons_id, "variation_month"] < 0.085).astype(float)
    
    cons_info.loc[cons_id, "pq_z_max"] = df["pq_z"].max()
    cons_info.loc[cons_id, "pq_z_min"] = df["pq_z"].min()
    
    cons_info.loc[cons_id, "entropy_month"] = entropy_df(df[["pq_f", "pq_g", "pq_p"]])
    cons_info.loc[cons_id, "std_month"] = std_df(df[["pq_f", "pq_g", "pq_p"]])
    
    cons_info.loc[cons_id, "diff_month"] = diff_median(df[["pq_f", "pq_g", "pq_p"]])
    cons_info.loc[cons_id, "devide_month"] = devide_median(df[["pq_f", "pq_g", "pq_p"]])    

    
cons_info

100%|████████████████████████████████████████████████████████████████████████████████| 8840/8840 [02:51<00:00, 51.64it/s]


,CONTRACT_CAP,RUN_CAP,SHIFT_NO,CHK_CYCLE,IS_FLAG,live_days,check_days,ELEC_TYPE_NAME_乡村居民生活用电,ELEC_TYPE_NAME_商业用电,ELEC_TYPE_NAME_城镇居民生活用电,ELEC_TYPE_NAME_学校教学和学生生活用电,ELEC_TYPE_NAME_居民生活用电,ELEC_TYPE_NAME_普通工业,ELEC_TYPE_NAME_非居民照明,ELEC_TYPE_NAME_非工业,VOLT_NAME_交流10kV,VOLT_NAME_交流220V,VOLT_NAME_交流380V,PRC_NAME_居民合表电价(1-10千伏）,PRC_NAME_居民合表电价(不满1千伏）,TMP_NAME_非临时用电,pq_z_mean,pq_z_median,pq_z_std,pq_z_std/mean,pq_z_entropy,pq_fg_distance,pq_gp_diatance,pq_fp_distance,pq_fg_cosine,pq_gp_cosine,pq_fp_cosine,pq_fz_cosine,pq_gz_cosine,pq_pz_cosine,variation_month,pq_z_mean_4500,variation_month_0085,pq_z_max,pq_z_min,entropy_month,std_month,diff_month,devide_month
ID,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
329465205,4.0,4.0,1.0,240.0,0,6040,6040,1,0,0,0,0,0,0,0,0,1,0,0,1,1,729.000000,740.0,352.359394,0.483346,2.074490,1100.455815,1368.865223,2425.555813,0.014260,1.030399,1.069618,0.001869,0.005819,1.054778,0.985564,0.0,0.0,1440,218,1.098612,236.742067,915.0,5.338216
329465245,4.0,4.0,1.0,240.0,0,6040,6040,1,0,0,0,0,0,0,0,0,1,0,0,1,1,1039.318182,996.5,345.823708,0.332741,1.652715,1884.471013,1641.545004,3495.833377,0.009833,1.049541,1.049255,0.001004,0.004561,1.048902,1.036198,0.0,0.0,2263,613,1.098612,356.826146,1349.5,6.739272
329465357,8.0,8.0,1.0,240.0,0,6040,6040,1,0,0,0,0,0,0,0,0,1,0,0,1,1,1712.863636,1656.0,434.369417,0.253593,1.360200,1005.509821,7926.527613,7997.614144,0.000281,0.949959,0.947601,0.711070,0.713365,0.028826,1.653016,0.0,0.0,3401,1186,0.678523,941.880880,3151.0,0.000000
329465360,2.0,2.0,1.0,240.0,0,6040,6040,1,0,0,0,0,0,0,0,0,1,0,0,1,1,1080.636364,1068.0,341.019975,0.315573,1.693263,2167.596826,1586.905164,3726.984304,0.010028,0.850390,0.860356,0.000896,0.004936,0.856495,1.078517,0.0,0.0,2240,553,1.098612,384.279868,1495.5,7.056769
329465439,8.0,8.0,1.0,240.0,0,6040,6040,1,0,0,0,0,0,0,0,0,1,0,0,1,1,1180.045455,1159.5,273.560036,0.231822,1.288252,0.000000,5675.094449,5675.094449,0.000000,1.000000,1.000000,1.000000,1.000000,0.000000,1.732051,0.0,0.0,2211,678,0.636514,681.299561,2319.0,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2862711473,39.0,39.0,0.0,120.0,0,759,759,1,0,0,0,0,0,0,0,0,0,1,0,1,1,1609.272727,1584.0,458.106474,0.284667,1.858519,2535.397208,2653.334506,5183.746811,0.000915,1.078510,1.073679,0.000104,0.000402,1.074872,0.991196,0.0,0.0,3044,808,1.098612,532.213416,2065.5,5.876878
2864274650,13.0,13.0,0.0,120.0,0,758,758,1,0,0,0,0,0,0,0,0,1,0,0,1,1,424.545455,98.5,451.592392,1.063708,1.306844,1174.220593,855.526154,2021.216218,0.005729,1.143152,1.123854,0.000500,0.002849,1.128561,0.998110,0.0,0.0,1670,53,1.098612,148.889645,122.0,5.550620
2872313739,39.0,39.0,0.0,120.0,0,743,743,1,0,0,0,0,0,0,0,0,0,1,0,1,1,0.636364,0.0,2.258222,3.548635,0.367649,8.944272,1.000000,9.848858,0.086188,1.000000,1.000000,0.000711,0.071523,1.000000,1.055087,0.0,0.0,10,0,0.078869,0.329195,0.0,0.000000


<h3>elec_day数据预处理</h3>

In [24]:
cons_info["kwh_mean"] = 0
cons_info["kwh_median"] = 0
cons_info["kwh_std"] = 0
cons_info["kwh_std/mean"] = 0
cons_info["kwh_entropy"] = 0

cons_info["kwh_pap_r23_distance"] = 0
cons_info["kwh_pap_r24_distance"] = 0
cons_info["kwh_pap_r34_distance"] = 0

cons_info["kwh_pap_r23_cosine"] = 0
cons_info["kwh_pap_r24_cosine"] = 0
cons_info["kwh_pap_r34_cosine"] = 0

cons_info["kwh_pap_r1_cosine"] = 0
cons_info["kwh_pap_r2_cosine"] = 0
cons_info["kwh_pap_r3_cosine"] = 0
cons_info["kwh_pap_r4_cosine"] = 0
cons_info["variation_day"] = 0

cons_info["kwh_max"] = 0
cons_info["kwh_min"] = 0

cons_info["entropy_day"] = 0
cons_info["std_day"] = 0

cons_info["diff_day"] = 0
cons_info["devide_day"] = 0

In [25]:
elec_day = pd.read_csv("../data/训练组_比特币挖矿_日用电明细（20211217）.csv")
elec_day

,id,rq,kwh,kwh_rap,kwh_pap_r1,kwh_pap_r2,kwh_pap_r3,kwh_pap_r4
0,329449398,2020-01-22 00:00:00,11.77,0.0,0.0,7.40,0.0,4.36
1,329449398,2020-01-23 00:00:00,16.54,0.0,0.0,8.68,0.0,7.86
2,329449398,2020-01-24 00:00:00,10.86,0.0,0.0,5.51,0.0,5.36
3,329449398,2020-01-25 00:00:00,10.91,0.0,0.0,6.48,0.0,4.43
4,329449398,2020-01-26 00:00:00,10.85,0.0,0.0,6.32,0.0,4.52
...,...,...,...,...,...,...,...,...
954564,2878334859,2021-10-05 00:00:00,7.22,NaN,0.0,4.49,0.0,2.73
954565,2878334859,2021-10-06 00:00:00,7.23,NaN,0.0,4.48,0.0,2.74
954566,2878334859,2021-10-07 00:00:00,7.20,NaN,0.0,4.48,0.0,2.73
954567,2878334859,2021-10-08 00:00:00,7.23,NaN,0.0,4.50,0.0,2.73


In [26]:
elec_day["rq"] = pd.to_datetime(elec_day["rq"])
elec_day

,id,rq,kwh,kwh_rap,kwh_pap_r1,kwh_pap_r2,kwh_pap_r3,kwh_pap_r4
0,329449398,2020-01-22,11.77,0.0,0.0,7.40,0.0,4.36
1,329449398,2020-01-23,16.54,0.0,0.0,8.68,0.0,7.86
2,329449398,2020-01-24,10.86,0.0,0.0,5.51,0.0,5.36
3,329449398,2020-01-25,10.91,0.0,0.0,6.48,0.0,4.43
4,329449398,2020-01-26,10.85,0.0,0.0,6.32,0.0,4.52
...,...,...,...,...,...,...,...,...
954564,2878334859,2021-10-05,7.22,NaN,0.0,4.49,0.0,2.73
954565,2878334859,2021-10-06,7.23,NaN,0.0,4.48,0.0,2.74
954566,2878334859,2021-10-07,7.20,NaN,0.0,4.48,0.0,2.73
954567,2878334859,2021-10-08,7.23,NaN,0.0,4.50,0.0,2.73


In [27]:
for cons_id in tqdm(cons_info.index):
    df = elec_day[elec_day["id"] == cons_id]
    df = df.drop(columns=["id"]).sort_values("rq").set_index("rq")
    df = df.fillna(method="ffill").fillna(method="bfill")
    cons_info.loc[cons_id, "kwh_mean"] = df["kwh"].mean()
    cons_info.loc[cons_id, "kwh_median"] = df["kwh"].median()
    cons_info.loc[cons_id, "kwh_std"] = df["kwh"].std()
    cons_info.loc[(cons_info.index == cons_id) & (cons_info["kwh_mean"] > 0), "kwh_std/mean"] = cons_info["kwh_std"] / cons_info["kwh_mean"]
    cons_info.loc[cons_id, "kwh_entropy"] = entropy(df["kwh"])

    cons_info.loc[cons_id, "kwh_pap_r23_distance"] = distance.euclidean(df["kwh_pap_r2"], df["kwh_pap_r3"])
    cons_info.loc[cons_id, "kwh_pap_r24_distance"] = distance.euclidean(df["kwh_pap_r2"], df["kwh_pap_r4"])
    cons_info.loc[cons_id, "kwh_pap_r34_distance"] = distance.euclidean(df["kwh_pap_r3"], df["kwh_pap_r4"])

    cons_info.loc[cons_id, "kwh_pap_r23_cosine"] = cosine(df["kwh_pap_r2"], df["kwh_pap_r3"])
    cons_info.loc[cons_id, "kwh_pap_r24_cosine"] = cosine(df["kwh_pap_r2"], df["kwh_pap_r4"])
    cons_info.loc[cons_id, "kwh_pap_r34_cosine"] = cosine(df["kwh_pap_r3"], df["kwh_pap_r4"])
    
    cons_info.loc[cons_id, "kwh_pap_r1_cosine"] = cosine(df["kwh_pap_r1"], df["kwh"])
    cons_info.loc[cons_id, "kwh_pap_r2_cosine"] = cosine(df["kwh_pap_r2"], df["kwh"])
    cons_info.loc[cons_id, "kwh_pap_r3_cosine"] = cosine(df["kwh_pap_r3"], df["kwh"])
    cons_info.loc[cons_id, "kwh_pap_r4_cosine"] = cosine(df["kwh_pap_r4"], df["kwh"])
    
    cons_info.loc[cons_id, "variation_day"] = variation(df[["kwh_pap_r2", "kwh_pap_r3", "kwh_pap_r4"]])
    
    cons_info.loc[cons_id, "kwh_max"] = df["kwh"].max()
    cons_info.loc[cons_id, "kwh_min"] = df["kwh"].min()
    
    cons_info.loc[cons_id, "entropy_day"] = entropy_df(df[["kwh_pap_r2", "kwh_pap_r3", "kwh_pap_r4"]])
    cons_info.loc[cons_id, "std_day"] = std_df(df[["kwh_pap_r2", "kwh_pap_r3", "kwh_pap_r4"]])
    
    cons_info.loc[cons_id, "diff_day"] = diff_median(df[["kwh_pap_r2", "kwh_pap_r3", "kwh_pap_r4"]])
    cons_info.loc[cons_id, "devide_day"] = devide_median(df[["kwh_pap_r2", "kwh_pap_r3", "kwh_pap_r4"]])
    
    
cons_info

100%|████████████████████████████████████████████████████████████████████████████████| 8840/8840 [04:47<00:00, 30.76it/s]


,CONTRACT_CAP,RUN_CAP,SHIFT_NO,CHK_CYCLE,IS_FLAG,live_days,check_days,ELEC_TYPE_NAME_乡村居民生活用电,ELEC_TYPE_NAME_商业用电,ELEC_TYPE_NAME_城镇居民生活用电,ELEC_TYPE_NAME_学校教学和学生生活用电,ELEC_TYPE_NAME_居民生活用电,ELEC_TYPE_NAME_普通工业,ELEC_TYPE_NAME_非居民照明,ELEC_TYPE_NAME_非工业,VOLT_NAME_交流10kV,VOLT_NAME_交流220V,VOLT_NAME_交流380V,PRC_NAME_居民合表电价(1-10千伏）,PRC_NAME_居民合表电价(不满1千伏）,TMP_NAME_非临时用电,pq_z_mean,pq_z_median,pq_z_std,pq_z_std/mean,pq_z_entropy,pq_fg_distance,pq_gp_diatance,pq_fp_distance,pq_fg_cosine,pq_gp_cosine,pq_fp_cosine,pq_fz_cosine,pq_gz_cosine,pq_pz_cosine,variation_month,pq_z_mean_4500,variation_month_0085,pq_z_max,pq_z_min,entropy_month,std_month,diff_month,devide_month,kwh_mean,kwh_median,kwh_std,kwh_std/mean,kwh_entropy,kwh_pap_r23_distance,kwh_pap_r24_distance,kwh_pap_r34_distance,kwh_pap_r23_cosine,kwh_pap_r24_cosine,kwh_pap_r34_cosine,kwh_pap_r1_cosine,kwh_pap_r2_cosine,kwh_pap_r3_cosine,kwh_pap_r4_cosine,variation_day,kwh_max,kwh_min,entropy_day,std_day,diff_day,devide_day
ID,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
329465205,4.0,4.0,1.0,240.0,0,6040,6040,1,0,0,0,0,0,0,0,0,1,0,0,1,1,729.000000,740.0,352.359394,0.483346,2.074490,1100.455815,1368.865223,2425.555813,0.014260,1.030399,1.069618,0.001869,0.005819,1.054778,0.985564,0.0,0.0,1440,218,1.098612,236.742067,915.0,5.338216,17.845278,17.115,10.812316,0.605892,2.155296,144.179175,82.996603,78.143830,1.0,0.112179,1.0,1.0,0.013884,1.0,0.048088,1.039291,45.71,0.66,1.060104,6.192106,20.875,3.454500e+08
329465245,4.0,4.0,1.0,240.0,0,6040,6040,1,0,0,0,0,0,0,0,0,1,0,0,1,1,1039.318182,996.5,345.823708,0.332741,1.652715,1884.471013,1641.545004,3495.833377,0.009833,1.049541,1.049255,0.001004,0.004561,1.048902,1.036198,0.0,0.0,2263,613,1.098612,356.826146,1349.5,6.739272,34.895556,35.400,7.727224,0.221439,2.111565,254.665742,143.991545,119.935441,1.0,0.042256,1.0,1.0,0.004351,1.0,0.019658,1.045283,54.73,17.88,1.098612,12.109005,47.185,7.074000e+08
329465357,8.0,8.0,1.0,240.0,0,6040,6040,1,0,0,0,0,0,0,0,0,1,0,0,1,1,1712.863636,1656.0,434.369417,0.253593,1.360200,1005.509821,7926.527613,7997.614144,0.000281,0.949959,0.947601,0.711070,0.713365,0.028826,1.653016,0.0,0.0,3401,1186,0.678523,941.880880,3151.0,0.000000,55.155648,55.575,10.426073,0.189030,2.046083,464.741106,348.667828,122.562718,1.0,0.039354,1.0,1.0,0.001687,1.0,0.024876,1.236592,79.75,26.31,1.090055,22.801373,88.530,1.329000e+09
329465360,2.0,2.0,1.0,240.0,0,6040,6040,1,0,0,0,0,0,0,0,0,1,0,0,1,1,1080.636364,1068.0,341.019975,0.315573,1.693263,2167.596826,1586.905164,3726.984304,0.010028,0.850390,0.860356,0.000896,0.004936,0.856495,1.078517,0.0,0.0,2240,553,1.098612,384.279868,1495.5,7.056769,34.803426,34.280,11.216622,0.322285,1.840667,268.892532,161.198520,114.334646,1.0,0.034102,1.0,1.0,0.002899,1.0,0.017558,1.103814,74.44,14.16,1.098612,12.624684,49.825,7.327500e+08
329465439,8.0,8.0,1.0,240.0,0,6040,6040,1,0,0,0,0,0,0,0,0,1,0,0,1,1,1180.045455,1159.5,273.560036,0.231822,1.288252,0.000000,5675.094449,5675.094449,0.000000,1.000000,1.000000,1.000000,1.000000,0.000000,1.732051,0.0,0.0,2211,678,0.636514,681.299561,2319.0,0.000000,34.382593,35.030,7.460780,0.216993,2.055828,262.847209,164.456730,106.217331,1.0,0.045007,1.0,1.0,0.003703,1.0,0.023050,1.094278,52.60,16.35,1.098612,12.514458,47.980,7.183500e+08
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2862711473,39.0,39.0,0.0,120.0,0,759,759,1,0,0,0,0,0,0,0,0,0,1,0,1,1,1609.272727,1584.0,458.106474,0.284667,1.858519,2535.397208,2653.334506,5183.746811,0.000915,1.078510,1.073679,0.000104,0.000402,1.074872,0.991196,0.0,0.0,3044,808,1.098612,532.213416,2065.5,5.876878,37.555278,34.750,17.699735,0.471298,2.198077,281.493464,142.254446,154.273303,1.0,0.046646,1.0,1.0,0.005853,1.0,0.019615,0.962473,74.60,13.68,1.085776,12.263838,42.910,6.667500e+08
286427

<h2>数据集划分</h2>

In [28]:
x = cons_info.drop(columns=["IS_FLAG"])
y = cons_info["IS_FLAG"]
print(x.shape, y.shape)

(8840, 65) (8840,)


<h2>标准化</h2>

In [29]:
ss = StandardScaler()
x = ss.fit_transform(x)
x

array([[-0.31260749, -0.31917163,  1.05812375, ...,  0.05176978,
         0.04269137,  0.55764591],
       [-0.31260749, -0.31917163,  1.05812375, ...,  0.30898424,
         0.34491997,  1.84521864],
       [-0.2464131 , -0.25086739,  1.05812375, ...,  0.77379383,
         0.81985884,  4.05645004],
       ...,
       [ 0.2665934 ,  0.27849043, -0.86811471, ..., -0.21740823,
        -0.19710423, -0.671231  ],
       [ 0.2665934 ,  0.27849043, -0.86811471, ..., -0.09737511,
        -0.10543626, -0.23794917],
       [ 0.11765603,  0.1248059 , -0.86811471, ...,  0.14671492,
         0.17812562,  1.06883309]])

<h1>做出来一个模型</h1>

smo = ADASYN(random_state=42, n_jobs=-1)
x, y = smo.fit_sample(x, y)

In [57]:
estimator = LGBMClassifier(
    objective="binary", 
    n_jobs=-1,
    learning_rate=0.01
)

In [58]:
model = BalancedBaggingClassifier(
    base_estimator=estimator,
    n_estimators=300,
    max_samples=1.0,
    max_features=1.0,
    bootstrap=True,
    bootstrap_features=True,
    n_jobs=12,
    random_state=42,
    verbose=1
)

In [59]:
model.fit(x, y)

[Parallel(n_jobs=12)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=12)]: Done   2 out of  12 | elapsed:    6.9s remaining:   34.7s
[Parallel(n_jobs=12)]: Done  12 out of  12 | elapsed:    7.2s finished


BalancedBaggingClassifier(base_estimator=LGBMClassifier(learning_rate=0.01,
                                                        objective='binary'),
                          bootstrap_features=True, n_estimators=300, n_jobs=12,
                          random_state=42, verbose=1)

cv = GridSearchCV(
    estimator=model,
    param_grid={
        "n_estimators": list(range(50, 700, 50)),
        "num_leaves": list(range(1, 50, 5)),
        "max_depth": [-1] + list(range(2, 11, 2)),
        "subsample_for_bin": list(range(100000, 700000, 100000))
    },
    scoring="f1",
    n_jobs=-1,
    iid=False,
    cv=10,
    verbose=1
)

try:
    model = load("../models/best_lightBGM.pkl")
    print("n_estimators", model.n_estimators)
    print("num_leaves", model.num_leaves)
    print("max_depth", model.max_depth)
    print("subsample_for_bin", model.subsample_for_bin)
    f1 = f1_score(y, model.predict(x))
except FileNotFoundError as e:
    cv.fit(x, y)
    model = cv.best_estimator_
    print(cv.best_params_)
    f1 = cv.best_score_
    model.fit(x, y)
    dump(model, "../models/best_lightBGM.pkl")

In [60]:
f1_score(y, model.predict(x))

[Parallel(n_jobs=12)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=12)]: Done   2 out of  12 | elapsed:    0.9s remaining:    4.8s
[Parallel(n_jobs=12)]: Done  12 out of  12 | elapsed:    2.5s finished


0.0

In [61]:
accuracy_score(y, model.predict(x))

[Parallel(n_jobs=12)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=12)]: Done   2 out of  12 | elapsed:    0.8s remaining:    4.3s
[Parallel(n_jobs=12)]: Done  12 out of  12 | elapsed:    2.5s finished


0.998868778280543

In [62]:
roc_auc_score(y, model.predict(x))

[Parallel(n_jobs=12)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=12)]: Done   2 out of  12 | elapsed:    0.7s remaining:    4.1s
[Parallel(n_jobs=12)]: Done  12 out of  12 | elapsed:    2.8s finished


0.5

In [63]:
features = list(cons_info.columns)
features.pop(features.index("IS_FLAG"))
for i, importance in enumerate(model.feature_importances_):
    print(features[i] + "\t" + str(importance))

AttributeError: 'BalancedBaggingClassifier' object has no attribute 'feature_importances_'

<h1>处理测试数据</h1>

<h2>数据预处理</h2>

<h3>档案数据预处理</h3>

In [ ]:
cons_info_test = pd.read_csv("../data/测试组_比特币挖矿_档案明细（20211220）.csv", index_col=0, quotechar='"', encoding="gbk")
cons_info_test

In [ ]:
cons_info_test = cons_info_test.drop(columns=["CANCEL_DATE", "TMP_DATE"])
cons_info_test

In [ ]:
cons_info_test["BUILD_DATE"] = pd.to_datetime(cons_info_test["BUILD_DATE"])
cons_info_test["LAST_CHK_DATE"] = pd.to_datetime(cons_info_test["LAST_CHK_DATE"])
cons_info_test

In [ ]:
cons_info_test["SHIFT_NO"] = cons_info_test["SHIFT_NO"].fillna(0)
cons_info_test

In [ ]:
cons_info_test["now"] = "2021-12-31"
cons_info_test["now"] = pd.to_datetime(cons_info_test["now"])
cons_info_test

In [ ]:
cons_info_test["live_days"] = (cons_info_test["now"] - cons_info_test["BUILD_DATE"]).map(lambda x: x.days)
cons_info_test["check_days"] = (cons_info_test["now"] - cons_info_test["LAST_CHK_DATE"]).map(lambda x: x.days)
cons_info_test

In [ ]:
cons_info_test = cons_info_test.drop(columns=["BUILD_DATE", "LAST_CHK_DATE", "now"])
cons_info_test

In [ ]:
cons_info_test.info()

In [ ]:
cons_info_test = pd.get_dummies(cons_info_test)
cons_info_test

In [ ]:
cons_info_test["CHK_CYCLE"] = cons_info_test["CHK_CYCLE"].fillna(cons_info_test["CHK_CYCLE"].median())
cons_info_test.info()

In [ ]:
cons_info_test["PRC_NAME_居民合表电价(1-10千伏）"] = cons_info_test["PRC_NAME_居民合表电价(1-10千伏）"] + cons_info_test["PRC_NAME_居民生活1_10kV(合表)"]
cons_info_test = cons_info_test.drop(columns=["PRC_NAME_居民生活1_10kV(合表)"])
cons_info_test

In [ ]:
cons_info_test = cons_info_test.rename(columns={"PRC_NAME_居民生活<1kV(合表)": "PRC_NAME_居民合表电价(不满1千伏）"})
cons_info_test

In [ ]:
cons_info_test["VOLT_NAME_交流10kV"] = cons_info_test["VOLT_NAME_交流10kV"] + cons_info_test["VOLT_NAME_交流6kV"]
cons_info_test = cons_info_test.drop(columns=["VOLT_NAME_交流6kV"])
cons_info_test

In [ ]:
cons_info_test["ELEC_TYPE_NAME_普通工业"] = cons_info_test["ELEC_TYPE_NAME_普通工业"] + cons_info_test["ELEC_TYPE_NAME_大工业用电"]
cons_info_test = cons_info_test.drop(columns=["ELEC_TYPE_NAME_大工业用电"])
cons_info_test["ELEC_TYPE_NAME_非工业"] = cons_info_test["ELEC_TYPE_NAME_非工业"] + cons_info_test["ELEC_TYPE_NAME_农业生产用电"]
cons_info_test = cons_info_test.drop(columns=["ELEC_TYPE_NAME_农业生产用电"])
cons_info_test

<h3>月电量处理</h3>

In [ ]:
cons_info_test["pq_z_mean"] = 0
cons_info_test["pq_z_median"] = 0
cons_info_test["pq_z_std"] = 0
cons_info_test["pq_z_std/mean"] = 0
cons_info_test["pq_z_entropy"] = 0
cons_info_test["pq_fg_distance"] = 0
cons_info_test["pq_gp_diatance"] = 0
cons_info_test["pq_fp_distance"] = 0
cons_info_test["pq_fg_cosine"] = 0
cons_info_test["pq_gp_cosine"] = 0
cons_info_test["pq_fp_cosine"] = 0
cons_info_test["pq_fz_cosine"] = 0
cons_info_test["pq_gz_cosine"] = 0
cons_info_test["pq_pz_cosine"] = 0
cons_info_test["variation_month"] = 0
cons_info_test["pq_z_mean_4500"] = 0
cons_info_test["variation_month_0085"] = 0

cons_info_test["pq_z_max"] = 0
cons_info_test["pq_z_min"] = 0

cons_info_test["entropy_month"] = 0
cons_info_test["std_month"] = 0

cons_info_test["diff_month"] = 0
cons_info_test["devide_month"] = 0

In [ ]:
elec_month_test = pd.read_csv("../data/测试组_比特币挖矿_月用电明细（20211217）.csv")
elec_month_test

In [ ]:
elec_month_test["ym"] = elec_month_test["ym"].astype(str)
elec_month_test

In [ ]:
for cons_id in tqdm(cons_info_test.index):
    df = elec_month_test[elec_month_test["id"] == cons_id]
    df = df.drop(columns=["id"]).sort_values("ym").set_index("ym")
    df = df.fillna(method="ffill").fillna(method="bfill")
    cons_info_test.loc[cons_id, "pq_z_mean"] = df["pq_z"].mean()
    cons_info_test.loc[cons_id, "pq_z_median"] = df["pq_z"].median()
    cons_info_test.loc[cons_id, "pq_z_std"] = df["pq_z"].std()
    cons_info_test.loc[(cons_info_test.index == cons_id) & (cons_info_test["pq_z_mean"] > 0), "pq_z_std/mean"] = cons_info_test["pq_z_std"] / cons_info_test["pq_z_mean"]
    cons_info_test.loc[cons_id, "pq_z_entropy"] = entropy(df["pq_z"])
    
    cons_info_test.loc[cons_id, "pq_fg_distance"] = distance.euclidean(df["pq_f"], df["pq_g"])
    cons_info_test.loc[cons_id, "pq_gp_diatance"] = distance.euclidean(df["pq_g"], df["pq_p"])
    cons_info_test.loc[cons_id, "pq_fp_distance"] = distance.euclidean(df["pq_f"], df["pq_p"])
    
    cons_info_test.loc[cons_id, "pq_fg_cosine"] = cosine(df["pq_f"], df["pq_g"])
    cons_info_test.loc[cons_id, "pq_gp_cosine"] = cosine(df["pq_g"], df["pq_p"])
    cons_info_test.loc[cons_id, "pq_fp_cosine"] = cosine(df["pq_f"], df["pq_p"])
    cons_info_test.loc[cons_id, "pq_fz_cosine"] = cosine(df["pq_f"], df["pq_z"])
    cons_info_test.loc[cons_id, "pq_gz_cosine"] = cosine(df["pq_g"], df["pq_z"])
    cons_info_test.loc[cons_id, "pq_pz_cosine"] = cosine(df["pq_p"], df["pq_z"])
    
    cons_info_test.loc[cons_id, "variation_month"] = variation(df[["pq_f", "pq_g", "pq_p"]])
    cons_info_test.loc[cons_id, "pq_z_mean_4500"] = (cons_info_test.loc[cons_id, "pq_z_mean"] > 4500).astype(float)
    cons_info_test.loc[cons_id, "variation_month_0085"] = (cons_info_test.loc[cons_id, "variation_month"] < 0.085).astype(float)

    cons_info_test.loc[cons_id, "pq_z_max"] = df["pq_z"].max()
    cons_info_test.loc[cons_id, "pq_z_min"] = df["pq_z"].min()
    
    cons_info_test.loc[cons_id, "entropy_month"] = entropy_df(df[["pq_f", "pq_g", "pq_p"]])
    cons_info_test.loc[cons_id, "std_month"] = std_df(df[["pq_f", "pq_g", "pq_p"]])
    
    cons_info_test.loc[cons_id, "diff_month"] = diff_median(df[["pq_f", "pq_g", "pq_p"]])
    cons_info_test.loc[cons_id, "devide_month"] = devide_median(df[["pq_f", "pq_g", "pq_p"]])  
    
    
cons_info_test

<h3>日电量处理</h3>

In [ ]:
cons_info_test["kwh_mean"] = 0
cons_info_test["kwh_median"] = 0
cons_info_test["kwh_std"] = 0
cons_info_test["kwh_std/mean"] = 0
cons_info_test["kwh_entropy"] = 0

cons_info_test["kwh_pap_r23_distance"] = 0
cons_info_test["kwh_pap_r24_distance"] = 0
cons_info_test["kwh_pap_r34_distance"] = 0

cons_info_test["kwh_pap_r23_cosine"] = 0
cons_info_test["kwh_pap_r24_cosine"] = 0
cons_info_test["kwh_pap_r34_cosine"] = 0

cons_info_test["kwh_pap_r1_cosine"] = 0
cons_info_test["kwh_pap_r2_cosine"] = 0
cons_info_test["kwh_pap_r3_cosine"] = 0
cons_info_test["kwh_pap_r4_cosine"] = 0
cons_info_test["variation_day"] = 0

cons_info_test["kwh_max"] = 0
cons_info_test["kwh_min"] = 0

cons_info_test["entropy_day"] = 0
cons_info_test["std_day"] = 0

cons_info_test["diff_day"] = 0
cons_info_test["devide_day"] = 0

In [ ]:
elec_day_test = pd.read_csv("../data/测试组_比特币挖矿_日用电明细（20211217）.csv")
elec_day_test

In [ ]:
elec_day_test["rq"] = pd.to_datetime(elec_day_test["rq"])
elec_day_test

In [ ]:
for cons_id in tqdm(cons_info_test.index):
    df = elec_day_test[elec_day_test["id"] == cons_id]
    if df.shape[0] == 0:
        continue
    df = df.drop(columns=["id"]).sort_values("rq").set_index("rq")
    df = df.fillna(method="ffill").fillna(method="bfill").fillna(0)
    cons_info_test.loc[cons_id, "kwh_mean"] = df["kwh"].mean()
    cons_info_test.loc[cons_id, "kwh_median"] = df["kwh"].median()
    cons_info_test.loc[cons_id, "kwh_std"] = df["kwh"].std()
    cons_info_test.loc[(cons_info_test.index == cons_id) & (cons_info_test["kwh_mean"] > 0), "kwh_std/mean"] = cons_info_test["kwh_std"] / cons_info_test["kwh_mean"]
    cons_info_test.loc[cons_id, "kwh_entropy"] = entropy(df["kwh"])

    cons_info_test.loc[cons_id, "kwh_pap_r23_distance"] = distance.euclidean(df["kwh_pap_r2"], df["kwh_pap_r3"])
    cons_info_test.loc[cons_id, "kwh_pap_r24_distance"] = distance.euclidean(df["kwh_pap_r2"], df["kwh_pap_r4"])
    cons_info_test.loc[cons_id, "kwh_pap_r34_distance"] = distance.euclidean(df["kwh_pap_r3"], df["kwh_pap_r4"])

    cons_info_test.loc[cons_id, "kwh_pap_r23_cosine"] = cosine(df["kwh_pap_r2"], df["kwh_pap_r3"])
    cons_info_test.loc[cons_id, "kwh_pap_r24_cosine"] = cosine(df["kwh_pap_r2"], df["kwh_pap_r4"])
    cons_info_test.loc[cons_id, "kwh_pap_r34_cosine"] = cosine(df["kwh_pap_r3"], df["kwh_pap_r4"])
    
    cons_info_test.loc[cons_id, "kwh_pap_r1_cosine"] = cosine(df["kwh_pap_r1"], df["kwh"])
    cons_info_test.loc[cons_id, "kwh_pap_r2_cosine"] = cosine(df["kwh_pap_r2"], df["kwh"])
    cons_info_test.loc[cons_id, "kwh_pap_r3_cosine"] = cosine(df["kwh_pap_r3"], df["kwh"])
    cons_info_test.loc[cons_id, "kwh_pap_r4_cosine"] = cosine(df["kwh_pap_r4"], df["kwh"])
    
    cons_info_test.loc[cons_id, "variation_day"] = variation(df[["kwh_pap_r2", "kwh_pap_r3", "kwh_pap_r4"]])
    
    cons_info_test.loc[cons_id, "kwh_max"] = df["kwh"].max()
    cons_info_test.loc[cons_id, "kwh_min"] = df["kwh"].min()
    
    cons_info_test.loc[cons_id, "entropy_day"] = entropy_df(df[["kwh_pap_r2", "kwh_pap_r3", "kwh_pap_r4"]])
    cons_info_test.loc[cons_id, "std_day"] = std_df(df[["kwh_pap_r2", "kwh_pap_r3", "kwh_pap_r4"]])
    
    cons_info_test.loc[cons_id, "diff_day"] = diff_median(df[["kwh_pap_r2", "kwh_pap_r3", "kwh_pap_r4"]])
    cons_info_test.loc[cons_id, "devide_day"] = devide_median(df[["kwh_pap_r2", "kwh_pap_r3", "kwh_pap_r4"]])
    
    
cons_info_test

In [ ]:
cons_info_test.info()

<h2>重新排列测试集</h2>

In [ ]:
cons_info_test = cons_info_test.reindex(columns=cons_info.columns).drop(columns=["IS_FLAG"])
cons_info_test.info()

In [ ]:
x_test = ss.transform(cons_info_test.values)
x_test.shape

In [ ]:
x_test

<h2>预测数据</h2>

In [ ]:
y_test = model.predict(x_test)
cons_info_test["label"] = y_test
cons_info_test

In [ ]:
cons_info_test.index.name = "id"
cons_info_test[["label"]].to_csv("../data/result20220223-temp.csv", index=True)